# KYC Funnel Analysis: Data Cleaning & Exploratory Analysis

Overview:
- Purpose: find root causes for a drop in overall pass rate and investigate a reported 'impossible state' bug.
- Steps: load data, clean and label common failure modes, create cohorts (esp. MEX + UNK), compute weekly metrics, and visualize trends and cohorts.
- Notes: added binary flags for common issues (fraud, quality, tech failures) to simplify aggregation and correlation analysis.

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import shap
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, TargetEncoder

from sklearn.metrics import roc_auc_score, classification_report

from sklearn.inspection import permutation_importance

In [ ]:
kyc_details = pd.read_csv('KYC_details.csv')
kyc_summary = pd.read_csv('KYC_summary.csv')

df = pd.merge(kyc_summary, kyc_details, on='user_reference', how='left')

In [ ]:
df['date_'] = pd.to_datetime(df['date_'])
df['is_pass'] = df['decision_type'].isin(['PASSED', 'APPROVED']).astype(int)

In [ ]:
anomalies = df[
    (df['decision_type'].isin(['PASSED', 'APPROVED'])) & 
    (df['watchlist_screening_decision'].isna())
]
df_clean = df.drop(anomalies.index).copy(deep=True)

In [ ]:
fraud_keywords = [
    'DIGITAL_COPY', 'MANIPULATED', 'FAKE', 'PUNCHED', 
    'MISMATCH_FRONT_BACK', 'PHOTOCOPY'
]
fraud_pattern = '|'.join(fraud_keywords)

df_clean['is_confirmed_fraud'] = (
    df_clean['image_checks_decision_details'].astype(str).str.contains(fraud_pattern, case=False, na=False) |
    df_clean['usability_decision_details'].astype(str).str.contains('PHOTOCOPY', case=False, na=False)
).astype(int)

In [ ]:
quality_keywords = [
    'GLARE', 'BLURRED', 'MISSING_PAGE', 'NOT_UPLOADED', 
    'DAMAGED_DOCUMENT', 'PART_OF_DOCUMENT_MISSING', 'PART_OF_DOCUMENT_HIDDEN',
    'BAD_QUALITY', 'FACE_NOT_FULLY_VISIBLE', 'liveness_UNDETERMINED'
]
quality_pattern = '|'.join(quality_keywords)

df_clean['is_quality_fail'] = (
    df_clean['usability_decision_details'].astype(str).str.contains(quality_pattern, case=False, na=False) |
    df_clean['liveness_decision_details'].astype(str).str.contains(quality_pattern, case=False, na=False)
).astype(int)

In [ ]:
df_clean['is_face_mismatch'] = (df_clean['similarity_decision_details'] == 'NO_MATCH').astype(int)
df_clean['is_unsupported_doc'] = df_clean['usability_decision_details'].isin(['UNSUPPORTED_DOCUMENT_TYPE', 'NOT_A_DOCUMENT']).astype(int)

In [ ]:
tech_fail_keywords = ['TECHNICAL_ERROR', 'MISMATCHING_DATAPOINTS', 'MISMATCH_HRZ_MRZ_DATA']
tech_pattern = '|'.join(tech_fail_keywords)

df_clean['is_tech_data_fail'] = (
    df_clean['data_checks_decision_details'].astype(str).str.contains(tech_pattern, case=False, na=False) |
    df_clean['extraction_decision_details'].astype(str).str.contains(tech_pattern, case=False, na=False)
).astype(int)

In [ ]:
bins = [0, 18, 25, 35, 45, 55, 100]
labels = ['<18', '18-25', '26-35', '36-45', '46-55', '55+']

df_clean['proxy_age'] = 2023 - pd.to_numeric(df_clean['year_birth'], errors='coerce')
df_clean['age_group'] = pd.cut(df_clean['proxy_age'], bins=bins, labels=labels).astype(str).replace('nan', 'Unknown')

In [ ]:
cols_to_fill = ['data_issuing_country', 'data_type', 'data_sub_type']
for col in cols_to_fill:
    df_clean[col] = df_clean[col].fillna('UNKOWN')

In [ ]:
df_clean['combo_country_type'] = df_clean['data_issuing_country'] + "_" + df_clean['data_type']
df_clean['combo_country_subtype'] = df_clean['data_issuing_country'] + "_" + df_clean['data_sub_type']
df_clean['combo_country_age'] = df_clean['data_issuing_country'] + "_" + df_clean['age_group']
df_clean['combo_type_age'] = df_clean['data_type'] + "_" + df_clean['age_group']

In [ ]:
def analyze_feature_groups(df, feature):
    stats = df.groupby(feature)['is_pass'].agg(['count', 'mean', 'sum']).reset_index()
    stats.columns = [feature, 'Total Attempts', 'Pass Rate', 'Passed Count']
    stats['Fail Count'] = stats['Total Attempts'] - stats['Passed Count']
    stats = stats.sort_values('Pass Rate', ascending=True)
    return stats

In [ ]:
group_stats_country_type = analyze_feature_groups(df_clean, 'combo_country_type')
meaningful_country_type = group_stats_country_type[group_stats_country_type['Total Attempts'] > 10]
meaningful_country_type.head(10)

In [ ]:
group_stats_country_age = analyze_feature_groups(df_clean, 'combo_country_age')
meaningful_country_age = group_stats_country_age[group_stats_country_age['Total Attempts'] > 10]
meaningful_country_age.head(10)

In [ ]:
df_clean['is_mex_unk'] = (
    (df_clean['data_issuing_country'] == 'MEX') & 
    (df_clean['data_type'] == 'UNKOWN')
).astype(int)

In [ ]:
agg_data = df_clean.groupby('week').agg({
    'is_pass': 'mean',
    'is_mex_unk': 'mean',
    'is_quality_fail': 'mean',
    'is_confirmed_fraud': 'mean',
    'is_face_mismatch': 'mean',
})

In [ ]:
correlation_matrix = agg_data.corr()['is_pass'].drop('is_pass').sort_values()
correlation_matrix

In [ ]:
normalized_weekly = (agg_data - agg_data.min()) / (agg_data.max() - agg_data.min())

plt.figure(figsize=(12, 6))
plt.plot(normalized_weekly.index, normalized_weekly['is_pass'], linewidth=4, color='black', label='Pass Rate')

top_suspect = "is_mex_unk"

plt.plot(
    normalized_weekly.index,
    normalized_weekly[top_suspect],
    linewidth=2, 
    linestyle='--',
    color="red",
    label=f'{top_suspect} (Corr: {correlation_matrix[top_suspect]:.2f})'
)

plt.title('Trend Analysis')
plt.legend()
plt.grid(True, alpha=0.3)

In [ ]:
df_clean['is_mex_unk_tech_fail'] = (
    (df_clean['is_mex_unk'] == 1) & 
    (df_clean['is_tech_data_fail'] == 1)
).astype(int)

df_clean['is_mex_unk_unssuported_document'] = (
    (df_clean['is_mex_unk'] == 1) & 
    (df_clean['is_unsupported_doc'] == 1)
).astype(int)

df_clean['is_mex_unk_quality_fail'] = (
    (df_clean['is_mex_unk'] == 1) & 
    (df_clean['is_quality_fail'] == 1)
).astype(int)

df_clean['is_mex_unk_mismatch'] = (
    (df_clean['is_mex_unk'] == 1) & 
    (df_clean['is_face_mismatch'] == 1)
).astype(int)

df_clean['is_mex_unk_fraud'] = (
    (df_clean['is_mex_unk'] == 1) & 
    (df_clean['is_confirmed_fraud'] == 1)
).astype(int)

In [ ]:
weekly_data_unkown = df_clean.groupby('week').agg({
    'is_pass': 'mean',
    'is_mex_unk_tech_fail': 'mean',
    'is_mex_unk_unssuported_document': 'mean',
    'is_mex_unk_quality_fail': 'mean',
    'is_mex_unk_mismatch': 'mean',
    'is_mex_unk_fraud': 'mean',
})

In [ ]:
correlation_matrix = weekly_data_unkown.corr()['is_pass'].drop('is_pass').sort_values()
correlation_matrix

In [ ]:
mex_unk_cohort = df_clean[
    (df_clean['is_mex_unk'] == 1)
].copy()

In [ ]:
mex_unk_cohort.groupby("usability_decision_details")["user_reference"].count()

In [ ]:
mex_unk_cohort_bugs = mex_unk_cohort[
    (mex_unk_cohort['usability_decision_details'] == 'OK')
]

mex_unk_cohort_bugs.groupby("image_checks_decision_details")["user_reference"].count()

In [ ]:
bug_cohort = df_clean[
    (df_clean['data_issuing_country'] == 'MEX') & 
    (df_clean['data_type'] == 'UNKOWN') & 
    (df_clean['usability_decision_details'] == 'OK')
].copy()

control_cohort = df_clean[
    (df_clean['data_issuing_country'] == 'MEX') & 
    (~(df_clean['data_type'] == 'UNKOWN')) & 
    (df_clean['usability_decision_details'] == 'OK')
].copy()

In [ ]:
bug_counts = bug_cohort['image_checks_decision_details'].value_counts()
control_counts = control_cohort['image_checks_decision_details'].value_counts()

In [ ]:
print(f"Total Users in Bug Cohort (MEX + UNK + Usability OK): {len(bug_cohort)}")
print("\nImage Checks Outcome for Bug Cohort:")
print(bug_counts)

print(f"\nTotal Users in Control Cohort (MEX + Known Doc + Usability OK): {len(control_cohort)}")
print("\nImage Checks Outcome for Control Cohort (Top 5):")
print(control_counts.head())

In [ ]:
bug_impossible_rate = (bug_counts.get('PRECONDITION_NOT_FULFILLED', 0) / len(bug_cohort)) * 100
control_impossible_rate = (control_counts.get('PRECONDITION_NOT_FULFILLED', 0) / len(control_cohort)) * 100

print(f"\n% of Users getting 'PRECONDITION FAIL' after 'USABILITY OK':")
print(f"  - Bug Cohort: {bug_impossible_rate:.2f}%")
print(f"  - Control Cohort: {control_impossible_rate:.2f}%")

In [ ]:
bug_cohort['is_impossible'] = (bug_cohort['image_checks_decision_details'] == 'PRECONDITION_NOT_FULFILLED').astype(int)
daily_bug_vol = bug_cohort.groupby('week')['is_impossible'].sum()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(daily_bug_vol.index, daily_bug_vol.values, color='red', marker='o', linewidth=2, label='Impossible Cases (Usability OK -> Precondition Fail)')
plt.title('Timeline of the "Impossible State" Bug (Daily Volume)')
plt.ylabel('Number of Users')
plt.xlabel('Date')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()

About the 'Impossible State' analysis:
- Definition: we flag an 
 when the issuing country is MEX, the detected document type is UNKOWN, and usability returned 'OK' — then later image checks produce a 'PRECONDITION_NOT_FULFILLED' (an impossible transition).
- This block computes weekly rates and plots pass rate vs the impossible-bug rate to show correlation and timing.
- Use this plot to correlate surges in this bug with drops in overall pass rate and to prioritize debugging.

In [ ]:
df_clean['is_impossible_bug'] = (
    (df_clean['data_issuing_country'] == 'MEX') & 
    (df_clean['data_type'] == 'UNKOWN') & 
    (df_clean['usability_decision_details'] == 'OK')
).astype(int)

daily_stats = df_clean.groupby('week').agg({
    'decision_type': lambda x: x.isin(['PASSED', 'APPROVED']).mean(),
    'is_impossible_bug': 'mean'
})
daily_stats.columns = ['Pass_Rate', 'Impossible_Bug_Rate']

daily_stats['Pass_Rate_Pct'] = daily_stats['Pass_Rate'] * 100
daily_stats['Impossible_Bug_Rate_Pct'] = daily_stats['Impossible_Bug_Rate'] * 100

fig, ax1 = plt.subplots(figsize=(14, 7))

color_pass = 'black'
ax1.set_xlabel('Date')
ax1.set_ylabel('Overall Pass Rate', color=color_pass, fontweight='bold', fontsize=12)
ax1.plot(daily_stats.index, daily_stats['Pass_Rate_Pct'], color=color_pass, linewidth=3, label='Pass Rate')
ax1.tick_params(axis='y', labelcolor=color_pass)
ax1.set_ylim(75, 90) 
ax1.grid(False)

ax2 = ax1.twinx()
color_bug = 'red'
ax2.set_ylabel('State Bug Rate', color=color_bug, fontweight='bold', fontsize=12)
ax2.plot(daily_stats.index, daily_stats['Impossible_Bug_Rate_Pct'], color=color_bug, linestyle='--', linewidth=2, marker='o', markersize=4, label='Impossible Bug (MEX UNK + Usability OK)')
ax2.tick_params(axis='y', labelcolor=color_bug)
ax2.set_ylim(3, 6)
ax2.grid(True, alpha=0.3)

plt.title('Pass Rate Drop vs. State Bug Surge', fontsize=16)
lines_1, labels_1 = ax1.get_legend_handles_labels()
lines_2, labels_2 = ax2.get_legend_handles_labels()
ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc='center left')

plt.tight_layout()